In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab\ Notebooks/VAE_CIFAR10

/content/drive/MyDrive/Colab Notebooks/VAE_CIFAR10


In [ ]:
import torch
import os
from torchvision.utils import save_image
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torchvision import datasets
real_data = datasets.CIFAR10(root="data", train=False, download=True, transform=transforms.ToTensor())
real_path = 'fid_real/'
os.makedirs(real_path, exist_ok=True)
for i in range(10000):
    save_image(real_data[i][0], f'{real_path}/{i}.png')

#generate from single layer VAE
from vae import VAE as SingleVAE
model_single = SingleVAE().to(device)
model_single.load_state_dict(torch.load('models/vae/vae.pth'))
model_single.eval()

single_path = 'fid_single_vae/'
os.makedirs(single_path, exist_ok=True)
with torch.no_grad():
    for i in range(0, 10000, 128):
        z = torch.randn(128, 256).to(device)
        generated = model_single.decode(z)
        generated = (generated + 1) / 2
        for j in range(128):
            if i + j < 10000:
                save_image(generated[j], f'{single_path}/{i+j}.png')

#generate from hVAE2
from hvae_2 import VAE as HVAE
model_hvae = HVAE().to(device)
model_hvae.load_state_dict(torch.load('models/hvae2/hvae2.pth'))
model_hvae.eval()

hvae_path = 'fid_hvae/'
os.makedirs(hvae_path, exist_ok=True)
with torch.no_grad():
    for i in range(0, 10000, 128):
        generated = model_hvae.generate(128, device)
        generated = (generated + 1) / 2
        for j in range(128):
            if i + j < 10000:
                save_image(generated[j], f'{hvae_path}/{i+j}.png')

In [8]:
import torch
import os
from torchvision.utils import save_image
from hvae2_ab import VAE as HVAE_AB
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_ab = HVAE_AB().to(device)
model_ab.load_state_dict(torch.load('models/hvae2_ab/hvae2_ab.pth'))
model_ab.eval()

hvae_ab_path = 'fid_hvae2_ab/'
os.makedirs(hvae_ab_path, exist_ok=True)
with torch.no_grad():
    for i in range(0, 10000, 128):
        generated = model_ab.generate(128, device)
        generated = (generated + 1) / 2
        for j in range(128):
            if i + j < 10000:
                save_image(generated[j], f'{hvae_ab_path}/{i+j}.png')

In [ ]:
!pip install pytorch-fid

In [10]:
!python -m pytorch_fid fid_real fid_single_vae
!python -m pytorch_fid fid_real fid_hvae
!python -m pytorch_fid fid_real fid_hvae2_ab

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 257MB/s]
100% 200/200 [04:58<00:00,  1.49s/it]
100% 200/200 [04:53<00:00,  1.47s/it]
FID:  105.53614977977327
100% 200/200 [01:58<00:00,  1.69it/s]
100% 200/200 [03:11<00:00,  1.05it/s]
FID:  120.94833839242312
100% 200/200 [01:55<00:00,  1.74it/s]
100% 200/200 [01:55<00:00,  1.73it/s]
FID:  116.15973051507092
